In [1]:
import pandas as pd

In [4]:
# 1. Load datasets
# Read the patient and billing data from the CSV files
patients = pd.read_csv("Patient_Data.csv")
billing = pd.read_csv("Billing_Data.csv")
patients

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45
5,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00


In [5]:
billing

,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


In [6]:
# Display basic info about each dataset (column names, data types, nulls)
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [7]:
billing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes


In [8]:
# 3. Select relevant columns for billing analysis
patients_billing = patients[['PatientID', 'Department', 'Doctor', 'BillAmount']]
patients_billing

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN
5,101,Cardiology,Dr. Smith,5000.0


In [9]:
# 4. Drop unnecessary administrative columns (if they exist)
patients_cleaned = patients.drop(columns=['ReceptionistID', 'CheckInTime'], errors='ignore')
patients_cleaned

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN
5,101,Alice,Cardiology,Dr. Smith,5000.0


In [10]:
# 5. Group by Department to calculate total bill amount
# Summarizes total revenue for each department
department_total = patients_billing.groupby('Department')['BillAmount'].sum().reset_index()
department_total

,Department,BillAmount
0,Cardiology,16200.0
1,Dermatology,0.0
2,Neurology,0.0
3,Orthopedics,7500.0


In [11]:
# 6. Remove duplicate patients based on PatientID
patients_unique = patients_cleaned.drop_duplicates(subset=['PatientID'])
patients_unique

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


In [12]:
# 7. Handle missing BillAmount values
# Replace NaN values with the mean bill amount
mean_bill = patients_unique['BillAmount'].mean()
mean_bill

np.float64(6233.333333333333)

In [14]:
patients_unique['BillAmount'] = patients_unique['BillAmount'].fillna(mean_bill)
patients_unique

C:\Users\WELCOME\AppData\Local\Temp\ipykernel_5308\3627152328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patients_unique['BillAmount'] = patients_unique['BillAmount'].fillna(mean_bill)


,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000
1,102,Bob,Neurology,Dr. John,6233.333333
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000
3,104,David,Cardiology,Dr. Smith,6200.000000
4,105,Eva,Dermatology,Dr. Rose,6233.333333


In [17]:
# 9. Concatenate additional patient records (row-wise)
# Example: New patients admitted this week
new_patients = pd.DataFrame({
'PatientID': [2001, 2002],
'Department': ['Cardiology', 'Neurology'],
'Doctor': ['Dr. Smith', 'Dr. Rao'],
'BillAmount': [4500, 5200]
})
patients_final = pd.concat([patients_unique, new_patients], ignore_index=True)
patients_final

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000
1,102,Bob,Neurology,Dr. John,6233.333333
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000
3,104,David,Cardiology,Dr. Smith,6200.000000
4,105,Eva,Dermatology,Dr. Rose,6233.333333
5,2001,NaN,Cardiology,Dr. Smith,4500.000000
6,2002,NaN,Neurology,Dr. Rao,5200.000000


In [20]:
# 10. Concatenate new billing-related columns (column-wise)
# Example: Adding insurance coverage and final billed amount after discounts

# Ensure the list length matches the number of rows in patients_final
insurance_values = [True, False] * (len(patients_final) // 2 + 1)  # Repeating to match length
insurance_values = insurance_values[:len(patients_final)]  # Trimming if needed
insurance_values

[True, False, True, False, True, False, True]

In [23]:
new_cols = pd.DataFrame({
    'InsuranceCovered': insurance_values,
    'FinalAmount': patients_final['BillAmount'] * 0.8  # Assuming 20% covered by insurance
})
new_cols

,InsuranceCovered,FinalAmount
0,True,4000.000000
1,False,4986.666667
2,True,6000.000000
3,False,4960.000000
4,True,4986.666667
5,False,3600.000000
6,True,4160.000000


In [24]:
# Concatenate new columns with the existing patient data
final_dataset = pd.concat([patients_final.reset_index(drop=True), new_cols], axis=1)

# Display the final dataset
print(final_dataset.head())

   PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000              True   
1        102      Bob    Neurology   Dr. John  6233.333333             False   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000              True   
3        104    David   Cardiology  Dr. Smith  6200.000000             False   
4        105      Eva  Dermatology   Dr. Rose  6233.333333              True   

   FinalAmount  
0  4000.000000  
1  4986.666667  
2  6000.000000  
3  4960.000000  
4  4986.666667  
